# Numpy deep neural network

### Imports

In [49]:
# set the file name (required)
__file__ = 'Numpy deep neural network.ipynb'

import numpy as np
from pprint import pprint
from IPython.display import Image

import ipytest.magics
import pytest

### Network architecture

![Network architecture](./supporting_visualizations/nn_architecture.png)

### Settings

In [59]:
NN_ARCHITECTURE = [
    {"input_dim": 2, "output_dim": 4, "activation": "relu"},
    {"input_dim": 4, "output_dim": 6, "activation": "relu"},
    {"input_dim": 6, "output_dim": 6, "activation": "relu"},
    {"input_dim": 6, "output_dim": 4, "activation": "relu"},
    {"input_dim": 4, "output_dim": 1, "activation": "sigmoid"},
]

### Activation functions

![Activations](./supporting_visualizations/activations.png)

In [80]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

### Initiation of parameter values for each layer

In [84]:
def initialize_parameters(nn_architecture, seed = 99):
    # random seed initiation
    np.random.seed(seed)
    # number of layers in our neural network
    number_of_layers = len(nn_architecture)
    # parameters storage initiation
    params_values = {}
    
    # iteration over network layers
    for idx, layer in enumerate(nn_architecture):
        # we number network layers from 1
        layer_idx = idx + 1
        
        # extracting the number of units in layers
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size) * 0.01
        params_values['b' + str(layer_idx)] = np.zeros(
            (layer_output_size, 1))
        
    return params_values

![Parameters sizes](./supporting_visualizations/params_sizes.png)

In [62]:
%%run_pytest[clean] -qq
# TEST PARAMETERS SHAPES

params_values = initialize_parameters(NN_ARCHITECTURE)

def test_first_layer_W_shape():
    assert params_values["W1"].shape == (NN_ARCHITECTURE[0]["output_dim"], NN_ARCHITECTURE[0]["input_dim"])
def test_first_layer_b_shape():
    assert params_values["b1"].shape == (NN_ARCHITECTURE[0]["output_dim"], 1)
def test_first_layer_W_shape():
    assert params_values["W2"].shape == (NN_ARCHITECTURE[1]["output_dim"], NN_ARCHITECTURE[1]["input_dim"])
def test_first_layer_b_shape():
    assert params_values["b2"].shape == (NN_ARCHITECTURE[1]["output_dim"], 1)

..                                                                       [100%]


### Single layer forward propagation step

![Single unit](./supporting_visualizations/single_unit.png)

In [65]:
def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
    # calculation of the input value for the activation function
    Z_curr = np.dot(W_curr, A_prev) + b_curr
    
    # selection of activation function
    if activation is "relu":
        activation_func = relu
    elif activation is "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')
        
    return activation_func(Z_curr), Z_curr

![Matrix sizes](./supporting_visualizations/matrix_sizes.png)

![Matrix sizes 2](./supporting_visualizations/matrix_sizes_2.png)

In [79]:
%%run_pytest[clean] -qq
# TEST OUTPUT FOR SINGLE LAYER FORWARD STEP

np.random.seed(2)
A_prev = np.random.randn(3,2)
W_curr = np.random.randn(1,3)
b_curr = np.random.randn(1,1)

A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu")

def test_relu_Z_shape():
    assert Z_curr.shape == (1,2)
def test_relu_A_shape():
    assert A_curr.shape == (1,2)
def test_relu_Z_value():
    assert np.allclose(Z_curr, np.array([[ 3.43896131, -2.08938436]]))
def test_relu_A_value():
    assert np.allclose(A_curr, np.array([[3.43896131, 0.        ]]))

....                                                                     [100%]


### Full forward propagation

In [129]:
def full_forward_propagation(X, params_values, nn_architecture):
    # creating a temporary memory to store the information needed for a backward step
    memory = {}
    # X vector is the activation for layer 0 
    A_curr = X
    
    # iteration over network layers
    for idx, layer in enumerate(nn_architecture):
        # we number network layers from 1
        layer_idx = idx + 1
        # transfer the activation from the previous iteration
        A_prev = A_curr
        
        # extraction of the activation function for the current layer
        activ_function_curr = layer["activation"]
        # extraction of W for the current layer
        W_curr = params_values["W" + str(layer_idx)]
        # extraction of b for the current layer
        b_curr = params_values["b" + str(layer_idx)]
        
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
        
    return A_curr, memory

In [130]:
np.random.seed(6)
X = np.random.randn(5, 4)
W1 = np.random.randn(4, 5)
b1 = np.random.randn(4, 1)
W2 = np.random.randn(3, 4)
b2 = np.random.randn(3, 1)
W3 = np.random.randn(1, 3)
b3 = np.random.randn(1, 1)

params_values = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
                "b3": b3}

In [131]:
nn_architecture = [
    {"input_dim": 4, "output_dim": 5, "activation": "relu"},
    {"input_dim": 4, "output_dim": 3, "activation": "relu"},
    {"input_dim": 3, "output_dim": 1, "activation": "sigmoid"},
]

In [132]:
A, memory = full_forward_propagation(X, params_values, nn_architecture)

### Calculating the value of the cost function

In [142]:
def get_cost_value(Y_hat, Y):
    # number of examples
    m = Y_hat.shape[1]
    # calculation of the cost according to the formula
    cost = -1 / m * (np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))
    return np.squeeze(cost)

In [141]:
Y = np.asarray([[1, 1, 1]])
Y_hat = np.array([[.8, .9, 0.4]])
get_cost_value(Y_hat, Y)

array(0.4149316)

In [146]:
def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    # number of examples
    m = A_prev.shape[1]
    
    # selection of activation function
    if activation is "relu":
        backward_activation_func = relu_backward
    elif activation is "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
        
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    
    # derivative of the matrix W
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    # derivative of the vector b
    db_curr = np.squeeze(np.sum(dZ_curr, axis=1, keepdims=True)) / m
    # derivative of the matrix A_prev
    dA_prev = np.dot(W_curr.T, dZ_curr)
    
    return dA_prev, dW_curr, db_curr

In [147]:
np.random.seed(2)
dA_curr = np.random.randn(1, 2)
A_prev = np.random.randn(3, 2)
W_curr = np.random.randn(1, 3)
b_curr = np.random.randn(1, 1)
Z_curr = np.random.randn(1, 2)

single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu")

(array([[ 0.44090989,  0.        ],
        [ 0.37883606,  0.        ],
        [-0.2298228 ,  0.        ]]),
 array([[ 0.44513824,  0.37371418, -0.10478989]]),
 -0.2083789237027353)